# Exercise 3: Image Smoothing & Edge Magnitude (NumPy-only)

## Problem Statement
Using **only NumPy**, on a synthetic 2D grayscale image:
1. Apply a 3×3 mean (box) filter using `sliding_window_view` (no SciPy).
2. Compute Sobel edge responses `Gx`, `Gy` with 3×3 kernels via convolution implemented in NumPy.
3. Compute edge magnitude `sqrt(Gx^2 + Gy^2)` and basic stats (min/max/mean).
4. Compare a naive Python-loop convolution with the vectorized version for a small image to observe speed difference.

## Approach
- Generate an image `img` of shape `(256, 256)` with gradients + noise.
- Use `np.pad` and `np.lib.stride_tricks.sliding_window_view` to build vectorized 2D conv.
- Define Sobel kernels and compute convolution via elementwise multiply and sum over the last two axes.

## Deliverables
- Print shapes, summary stats for smoothed image and edge magnitude, and timing comparison.

In [1]:
import numpy as np
from time import perf_counter

# ----- A) Synthetic image -----
rng = np.random.default_rng(7)
H, W = 256, 256
y = np.linspace(0, 1, H, dtype=np.float32)[:, None]
x = np.linspace(0, 1, W, dtype=np.float32)[None, :]
base = 0.6 * x + 0.4 * y  # smooth gradient
noise = rng.normal(0, 0.05, size=(H, W)).astype(np.float32)
img = np.clip(base + noise, 0, 1).astype(np.float32)

# ----- B) Vectorized 2D convolution using sliding windows -----
def conv2d_valid(img, kernel):
    kH, kW = kernel.shape
    # build sliding windows (valid conv, so no pad here)
    win = np.lib.stride_tricks.sliding_window_view(img, (kH, kW))  # shape: (H-kH+1, W-kW+1, kH, kW)
    # elementwise multiply and sum over last two axes
    return np.tensordot(win, kernel[::-1, ::-1], axes=([2,3],[0,1]))  # flip kernel for conv

# 3x3 mean (box) kernel
box = np.ones((3,3), dtype=np.float32) / 9.0
# For 'same' output, pad the image
pad = ((1,1), (1,1))
img_padded = np.pad(img, pad, mode='reflect')
smoothed = conv2d_valid(img_padded, box)

# ----- C) Sobel edges -----
Kx = np.array([[1,0,-1],[2,0,-2],[1,0,-1]], dtype=np.float32)
Ky = np.array([[1,2,1],[0,0,0],[-1,-2,-1]], dtype=np.float32)
gx = conv2d_valid(img_padded, Kx)
gy = conv2d_valid(img_padded, Ky)
edge_mag = np.sqrt(gx*gx + gy*gy)

print('smoothed shape:', smoothed.shape, 'edge_mag shape:', edge_mag.shape)
print('smoothed stats: min={:.4f} max={:.4f} mean={:.4f}'.format(smoothed.min(), smoothed.max(), smoothed.mean()))
print('edge_mag stats: min={:.4f} max={:.4f} mean={:.4f}'.format(edge_mag.min(), edge_mag.max(), edge_mag.mean()))

# ----- D) Naive loop conv vs vectorized (smaller image for fairness) -----
small = img[:64, :64]
kernel = box
# vectorized (with pad same) on small
s_pad = np.pad(small, ((1,1),(1,1)), mode='reflect')
t0 = perf_counter()
vec_out = conv2d_valid(s_pad, kernel)
t1 = perf_counter()

# naive loops
t2 = perf_counter()
out_loop = np.zeros_like(vec_out)
for i in range(vec_out.shape[0]):
    for j in range(vec_out.shape[1]):
        window = s_pad[i:i+3, j:j+3]
        out_loop[i, j] = np.sum(window * kernel[::-1, ::-1])
t3 = perf_counter()

print('\nTiming on 64x64 image with 3x3 box:')
print(f'  Vectorized: {(t1 - t0):.6f} s')
print(f'  Loop:       {(t3 - t2):.6f} s')
print('Max abs diff (vec vs loop):', float(np.max(np.abs(vec_out - out_loop))))

smoothed shape: (256, 256) edge_mag shape: (256, 256)
smoothed stats: min=0.0044 max=0.9954 mean=0.4999
edge_mag stats: min=0.0000 max=0.8440 mean=0.2151

Timing on 64x64 image with 3x3 box:
  Vectorized: 0.000467 s
  Loop:       0.041608 s
Max abs diff (vec vs loop): 2.9802322387695312e-08


Here’s a clear, section-by-section explanation of what the code does, the shapes you get at each step, and why these NumPy tricks work.

---

# Imports

```python
import numpy as np
from time import perf_counter
```

* `numpy` powers array math and the vectorized convolution.
* `perf_counter()` is a high-resolution timer to compare vectorized vs loop performance.

---

# A) Build a synthetic grayscale image

```python
rng = np.random.default_rng(7)
H, W = 256, 256
y = np.linspace(0, 1, H, dtype=np.float32)[:, None]   # (256, 1)
x = np.linspace(0, 1, W, dtype=np.float32)[None, :]   # (1, 256)
base = 0.6 * x + 0.4 * y                               # (256, 256) smooth gradient
noise = rng.normal(0, 0.05, size=(H, W)).astype(np.float32)
img = np.clip(base + noise, 0, 1).astype(np.float32)   # (256, 256)
```

**What’s happening**

* `x` and `y` are 1-D ramps broadcast into a 2-D plane:

  * `y` increases top→bottom; `x` increases left→right.
* `base = 0.6*x + 0.4*y` blends horizontal and vertical gradients to create a smooth image.
* `noise` adds small Gaussian noise (σ = 0.05) for realism.
* `clip` confines pixel values to the valid range `[0, 1]`.
* Final image `img` is `float32` of shape `(256, 256)`.

**Why `[:, None]` and `[None, :]`?**

* They turn 1-D arrays into column/row vectors so broadcasting yields a full 2-D grid without `meshgrid`.

---

# B) Vectorized 2D convolution (valid mode) with sliding windows

```python
def conv2d_valid(img, kernel):
    kH, kW = kernel.shape
    win = np.lib.stride_tricks.sliding_window_view(img, (kH, kW))
    return np.tensordot(win, kernel[::-1, ::-1], axes=([2,3],[0,1]))
```

**Key ideas**

* **sliding\_window\_view** builds a view of all `kH×kW` patches:

  * If `img` is `(H, W)` and kernel is `(kH, kW)`, then
    `win` has shape `((H-kH+1), (W-kW+1), kH, kW)`.
  * It’s a *view*, not a copy (uses strides) → memory-efficient.
* **Convolution vs correlation**:

  * True convolution flips the kernel; correlation does not.
  * `kernel[::-1, ::-1]` flips vertically and horizontally for convolution.
* **tensordot** multiplies each window by the flipped kernel and sums over the last two axes `(kH, kW)`, returning a 2-D array with shape `(H-kH+1, W-kW+1)`.

**Why “valid” mode?**

* No padding inside this function; it only returns outputs where the kernel fully overlaps the image.

---

# Box blur (“same” output via padding)

```python
box = np.ones((3,3), dtype=np.float32) / 9.0
pad = ((1,1), (1,1))
img_padded = np.pad(img, pad, mode='reflect')  # (258, 258)
smoothed = conv2d_valid(img_padded, box)       # (256, 256)
```

**What’s happening**

* The 3×3 **mean (box) filter** averages each pixel with its neighbors.
* To keep the **same** output size as the input, we pad by 1 pixel on all sides before doing a **valid** convolution:

  * Padding: reflect (mirror) the border, which avoids artificial dark/light rims typical of zero-padding.
  * After valid conv on `(258, 258)` with `(3, 3)`, you get `(256, 256)`.

---

# C) Sobel edge detection

```python
Kx = np.array([[1,0,-1],[2,0,-2],[1,0,-1]], dtype=np.float32)
Ky = np.array([[1,2,1],[0,0,0],[-1,-2,-1]], dtype=np.float32)
gx = conv2d_valid(img_padded, Kx)  # horizontal edges
gy = conv2d_valid(img_padded, Ky)  # vertical edges
edge_mag = np.sqrt(gx*gx + gy*gy)
```

**What’s happening**

* **Sobel** filters are 3×3 kernels that approximate intensity derivatives:

  * `Kx` emphasizes horizontal changes (detects **vertical** edges).
  * `Ky` emphasizes vertical changes (detects **horizontal** edges).
* We again use reflected padding to get an output the same size as `img`.
* `edge_mag` is the gradient magnitude $\sqrt{G_x^2 + G_y^2}$, combining both directions.

---

# Quick sanity stats

```python
print('smoothed shape:', smoothed.shape, 'edge_mag shape:', edge_mag.shape)
print('smoothed stats: min={:.4f} max={:.4f} mean={:.4f}'.format(smoothed.min(), smoothed.max(), smoothed.mean()))
print('edge_mag stats: min={:.4f} max={:.4f} mean={:.4f}'.format(edge_mag.min(), edge_mag.max(), edge_mag.mean()))
```

* Confirms output sizes are `(256, 256)`.
* Typical results: smoothed image has reduced variance; edge magnitude is mostly small with higher values at edges.

---

# D) Vectorized vs naive loop (performance + correctness)

```python
small = img[:64, :64]
kernel = box
s_pad = np.pad(small, ((1,1),(1,1)), mode='reflect')
t0 = perf_counter()
vec_out = conv2d_valid(s_pad, kernel)
t1 = perf_counter()
```

* Vectorized convolution on a **smaller** image `(64, 64)` (after padding: `(66, 66)`), returning `(64, 64)`.
* `t1 - t0` measures the fast path.

```python
t2 = perf_counter()
out_loop = np.zeros_like(vec_out)
for i in range(vec_out.shape[0]):
    for j in range(vec_out.shape[1]):
        window = s_pad[i:i+3, j:j+3]
        out_loop[i, j] = np.sum(window * kernel[::-1, ::-1])
t3 = perf_counter()
```

* **Naive** double loop that extracts each 3×3 window and computes the sum manually.
* Functionally identical to the vectorized convolution (also flips the kernel to remain a true convolution).
* `t3 - t2` is the slow path time.

```python
print('\\nTiming on 64x64 image with 3x3 box:')
print(f'  Vectorized: {(t1 - t0):.6f} s')
print(f'  Loop:       {(t3 - t2):.6f} s')
print('Max abs diff (vec vs loop):', float(np.max(np.abs(vec_out - out_loop))))
```

* You’ll typically see **large speedups** (often tens to hundreds of times) for the vectorized version.
* `Max abs diff` should be \~0 (floating-point rounding may produce tiny differences but here it’s exact because both paths do the same arithmetic order on small windows).

---

## Subtle but important details

* **Convolution vs correlation**
  Many “convolution” snippets actually compute correlation (no kernel flip). Here, we **flip** the kernel (`[::-1, ::-1]`) to implement true convolution.

* **sliding\_window\_view is a view**
  It creates an array with **strides**—no data copy. That makes it fast and memory-efficient, but be aware: the resulting view can be large for big kernels/images.

* **Padding choice impacts edges**

  * `reflect` mirrors values at boundaries (good for natural images).
  * `constant` (zeros) would darken the borders; `edge` repeats edge pixels; `wrap` is toroidal.

* **Data types**
  Using `float32` halves memory vs `float64` and is plenty for image filters. If you chain many ops or need extra precision, consider `float64`.

* **Complexity**

  * Vectorized conv (with `sliding_window_view` + `tensordot`): roughly $O(H \times W \times kH \times kW)$ but executed in compiled C and leveraging contiguous memory access.
  * Python loops have the same theoretical complexity but enormous interpreter overhead.

---

## How to extend

* Swap `box` for a **Gaussian** kernel (still 3×3 or 5×5).
* Add **non-maximum suppression** and **thresholding** to turn `edge_mag` into a binary edge map.
* Benchmark larger images or kernels; or implement **“valid/same/full”** modes generically inside `conv2d` by handling padding internally.
